In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import Ridge

from os import listdir
from os.path import isfile, join

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# This function removes highly correlated variables within the dataset. The function looks at the 
# Pearson Correlation Coefficient between every pair of variables in the dataset and returns all 
# of those between .8 and .999. While working with data from different companies, I found that by
# removing the first 90% of correlated variables within the DataFrame, in almost all cases, there
# were no longer correlations within the dataset.

def remove_corrs(df):
    
    # create a DataFrame with all the of the correlation coefficients, and then stack the pairings.
    corrs = df.corr().stack().reset_index()
    corrs.columns = ['1','2','R2']
    # create a temoporary DataFrame that contains all the pairs of variables that have a correlation
    # between .8 and .999. Because the .corr() method returns the pairings in both directions, I only
    # look at half of the cells because of duplicates.
    temp = corrs[(corrs.R2 > .8) & (corrs.R2 < .999)].sort_values('R2', ascending = False).reset_index(drop = True)
    correlations = temp[temp.index % 2 == 0]
    
    # look at how many times each variable shows up in the correlation column
    corr_index = correlations['1'].value_counts().index
    # delete the first 90% of the variables. This generally removes the columns that are most correlated with the others. 
    to_drop = round(len(corr_index)*.9)
    
    df2 = df.drop(corr_index[:to_drop], axis = 1)
    
    return df2

In [3]:
# This function preprocesses my data so it is ready for distanced based regressions.

def preprocess_data(df):
    
    df.index = pd.to_datetime(df.index)
    
    # Here, I shift all the independent variables back one day in order to make sure there is
    # no data leakage. Because we're trying to predict closing price, we can't know many of
    # things that happen throughout the day we're trying to predict.
    X = df.shift(1).dropna()
    
    # This is the function described above.
    X_data = remove_corrs(X)

    # Because I am running a regression, I can use a continuous variable as my dependent variable.
    # Ideally, my algorithim can predict the closing price at the end of the day with accuracy.
    y = df['adjustedclose'].iloc[1:]
    
    # Do a train test split with the first 80% of the data being the training set and the last 20%
    # as the testing set.
    train_num = round(len(X)*.8)
    test_num = round(len(X)*.2)
    print(train_num, test_num)

    X_train = X_data.iloc[:train_num]
    X_test = X_data.iloc[-test_num:]
    y_train = y.iloc[:train_num]
    y_test = y.iloc[-test_num:]
    
    # Because I am doing distance based regressions, I need to scale the data so that variables with
    # higher absolute values don't dominate the metrics.
    ss = StandardScaler()
    X_train_scaled = ss.fit_transform(X_train)
    X_test_scaled = ss.transform(X_test)

    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns).set_index(X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns).set_index(X_test.index)
    
    return X_train, X_test, y_train, y_test

In [28]:
def EN_test(X_train, X_test, y_train, y_test, cv=5):
    
    # Set GridSearchCV hyperparameters to compare & select
    grid = {
    'l1_ratio': [.01,.25,.5,.75,1],
    'alpha': [.01,.25,.5,.75,1],
    }
    
    # Instantiate & fit RidgeRegression
    en = ElasticNet(random_state = 42, max_iter = 10e5)
    
    # Instantiate & fit GridSearchCV with accuracy scoring
    gs = GridSearchCV(estimator=en, param_grid=grid, cv=cv, scoring='neg_root_mean_squared_error', n_jobs = -1)
    gs.fit(X_train, y_train)
    
    # Create prediction variable using test data
    y_hat_train = gs.predict(X_train)
    y_hat_test = gs.predict(X_test)
    
    # Run cross-validate score with cv folds from function parameter
    cv_results = cross_val_score(gs, X_train, y_train, cv=cv, scoring = 'neg_root_mean_squared_error')

    # Score the train and test sets.
    train_score = gs.score(X_train, y_train)
    test_score = gs.score(X_test, y_test)
    
    print(f'Train RMSE: {train_score}')
    print(f'Mean Cross-Val Score: {cv_results.mean()}')
    print(f'Test RMSE: {test_score}')

    # Create a DataFrame that has the predictions for the given days of the train set.
    preds_train = pd.concat([y_train, pd.DataFrame(y_hat_train, columns = ['predictions'], index = y_train.index), df.day_direction], axis = 1)
    preds_train.dropna(inplace = True)

    # Compare the direction of the predicted value with what actually happened on the given day.
    preds_train['direction'] = np.where(preds_train.predictions > preds_train.adjustedclose.shift(1), 1, 0)
    preds_train['correct'] = np.where(preds_train['direction'] == preds_train['day_direction'], 1, 0)
    train_accuracy = preds_train['correct'].value_counts(normalize = True)
    
    # Create a DataFrame that has the predictions for the given days of the test set.
    preds_test = pd.concat([y_test, pd.DataFrame(y_hat_test, columns = ['predictions'], index = y_test.index), df.day_direction], axis = 1)
    preds_test.dropna(inplace = True)

    # Compare the direction of the predicted value with what actually happend on the given day. 
    preds_test['direction'] = np.where(preds_test.predictions > preds_test.adjustedclose.shift(1), 1, 0)
    preds_test['correct'] = np.where(preds_test['direction'] == preds_test['day_direction'], 1, 0)
    test_accuracy = preds_test['correct'].value_counts(normalize = True)

    print(f'Train accuracy: {train_accuracy[1]}')
    print(f'Test accuracy: {test_accuracy[1]}')
    
    # Return the results in a list that can later be appended to a DataFrame.
    results = ['elastic net', train_score, cv_results.mean(), test_score, train_accuracy[1], test_accuracy[1]]
    
    return results

In [5]:
def SVR_test(X_train, X_test, y_train, y_test, cv=5):
    
    svr = LinearSVR(random_state = 42, max_iter = 10e5)
    svrmodel = svr.fit(X_train, y_train)
    
    train_score = svr.score(X_train, y_train)
    y_hat_train = svr.predict(X_train)
    train_rmse = mean_squared_error(y_train, y_hat_train, squared = False)
    train_cv_score = cross_val_score(svrmodel, X_train, y_train, scoring = 'neg_root_mean_squared_error').mean()
    
    test_score = svr.score(X_test, y_test)
    y_hat_test = svr.predict(X_test)
    test_rmse = mean_squared_error(y_test, y_hat_test, squared = False)
    
    preds_train = pd.concat([y_train, pd.DataFrame(y_hat_train, columns = ['predictions'], index = y_train.index), df.day_direction], axis = 1)
    preds_train.dropna(inplace = True)

    preds_train['direction'] = np.where(preds_train.predictions > preds_train.adjustedclose.shift(1), 1, 0)
    preds_train['correct'] = np.where(preds_train['direction'] == preds_train['day_direction'], 1, 0)
    train_accuracy = preds_train['correct'].value_counts(normalize = True)
    
    preds_test = pd.concat([y_test, pd.DataFrame(y_hat_test, columns = ['predictions'], index = y_test.index), df.day_direction], axis = 1)
    preds_test.dropna(inplace = True)

    preds_test['direction'] = np.where(preds_test.predictions > preds_test.adjustedclose.shift(1), 1, 0)
    preds_test['correct'] = np.where(preds_test['direction'] == preds_test['day_direction'], 1, 0)
    test_accuracy = preds_test['correct'].value_counts(normalize = True)
    
    print(f'Train RMSE : {train_rmse}')
    print(f'Mean Train Cross-Validation RMSE: {train_cv_score}')
    print(f'Test RMSE : {test_rmse}')
    print(f'Train accuracy: {train_accuracy[1]}')
    print(f'Test accuracy: {test_accuracy[1]}')
 
    results = ['SVR', train_score, train_cv_score, test_score, train_accuracy[1], test_accuracy[1]]
    
    return results

In [6]:
def Ridge_test(X_train, X_test, y_train, y_test, cv=5):
    
    # Set GridSearchCV hyperparameters to compare & select
    grid = {
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'],
    'alpha': [.01, .05, .1,.5,.75,1],
    }
    
    # Instantiate & fit RidgeRegression
    ridge = Ridge(random_state = 42, max_iter = 10e5)
    
    # Instantiate & fit GridSearchCV with accuracy scoring
    gs = GridSearchCV(estimator=ridge, param_grid=grid, cv=cv, scoring='r2', n_jobs = -1)
    gs.fit(X_train, y_train)
    
    # Return best hyperparameters
    ridge_params = gs.best_params_
    
    # Create prediction variable using test data
    y_hat_train = gs.predict(X_train)
    y_hat_test = gs.predict(X_test)

    # Run cross-validate score with cv folds from function parameter
    train_cv_score = cross_val_score(gs, X_train, y_train, cv=cv, scoring = 'neg_root_mean_squared_error').mean()
    
    # Run and print accuracy, recall, precision and f1 scores
    train_score = gs.score(X_train, y_train)
    test_score = gs.score(X_test, y_test)

    train_rmse = mean_squared_error(y_train, y_hat_train, squared = False)
    test_rmse = mean_squared_error(y_test, y_hat_test, squared = False)
    
    preds_train = pd.concat([y_train, pd.DataFrame(y_hat_train, columns = ['predictions'], index = y_train.index), df.day_direction], axis = 1)
    preds_train.dropna(inplace = True)

    preds_train['direction'] = np.where(preds_train.predictions > preds_train.adjustedclose.shift(1), 1, 0)
    preds_train['correct'] = np.where(preds_train['direction'] == preds_train['day_direction'], 1, 0)
    train_accuracy = preds_train['correct'].value_counts(normalize = True)
    
    preds_test = pd.concat([y_test, pd.DataFrame(y_hat_test, columns = ['predictions'], index = y_test.index), df.day_direction], axis = 1)
    preds_test.dropna(inplace = True)

    preds_test['direction'] = np.where(preds_test.predictions > preds_test.adjustedclose.shift(1), 1, 0)
    preds_test['correct'] = np.where(preds_test['direction'] == preds_test['day_direction'], 1, 0)
    test_accuracy = preds_test['correct'].value_counts(normalize = True)
    
    print(f'Train RMSE : {train_rmse}')
    print(f'Mean Train Cross-Validation RMSE: {train_cv_score}')
    print(f'Test RMSE : {test_rmse}')
    print(f'Train accuracy: {train_accuracy[1]}')
    print(f'Test accuracy: {test_accuracy[1]}')
 
    results = ['Ridge', train_rmse, train_cv_score, test_rmse, train_accuracy[1], test_accuracy[1]]
    
    return results

In [14]:
# get the files in the Formatted Data folder.
files = [f for f in listdir('.\FormattedData')]

# create a list of companies that can be analyzed.
companies = []
for i in files:
    company = i.split('.')[0]
    companies.append(company)

# create a list of random companies to analyze from within the formatted companies
test_companies = np.random.choice(companies, 10, replace = False)

In [15]:
test_companies

array(['GWW_formatted', 'FE_formatted', 'TYL_formatted', 'RSG_formatted',
       'DXCM_formatted', 'FN_formatted', 'FLT_formatted', 'ENS_formatted',
       'EA_formatted', 'TCYSF_formatted'], dtype='<U15')

In [16]:
# loop through the companies and perform the different alogrithims. Create a
# list of DataFrames that can then be used to compared results across different
# companies and algorithms.

results2 = []

for c in test_companies:
    csv_for_df = f'FormattedData/{c}.csv'
    df = pd.read_csv(csv_for_df, index_col = 0)
    X_train, X_test, y_train, y_test = preprocess_data(df)
    en_results = EN_test(X_train, X_test, y_train, y_test)
    svr_results = SVR_test(X_train, X_test, y_train, y_test)
    ridge_results = Ridge_test(X_train, X_test, y_train, y_test)
    c_results = pd.DataFrame([en_results, svr_results, ridge_results],
            columns = ['Model Type', 'Train RMSE', 'Cross-Val RMSE', 'Test RMSE', 
             'Train Accuracy', 'Test Accuracy'])
    c_results['company'] = c.split('_')[0]
    results2.append(c_results)

1798 450
Train RMSE: -25.37207247901399
Mean Cross-Val Score: -35.835940020819564
Test RMSE: -105.97096009516055
Train accuracy: 0.5116796440489433
Test accuracy: 0.45555555555555555
Train RMSE : 63.25160381567274
Mean Train Cross-Validation RMSE: -51.851581513962174
Test RMSE : 124.4006624102215
Train accuracy: 0.503337041156841
Test accuracy: 0.4622222222222222
Train RMSE : 23.07423823941355
Mean Train Cross-Validation RMSE: -47.36874928812652
Test RMSE : 92.37237142358589
Train accuracy: 0.517797552836485
Test accuracy: 0.46
1798 450
Train RMSE: -0.7576826509571929
Mean Cross-Val Score: -1.62700207745889
Test RMSE: -8.367635203412112
Train accuracy: 0.5261401557285873
Test accuracy: 0.5644444444444444
Train RMSE : 5.609375492288247
Mean Train Cross-Validation RMSE: -5.61503458570169
Test RMSE : 13.904028083671713
Train accuracy: 0.4916573971078977
Test accuracy: 0.5644444444444444
Train RMSE : 0.7222879680494303
Mean Train Cross-Validation RMSE: -1.576619672884296
Test RMSE : 9.0438

In [25]:
results
results2
best_test = pd.concat(results)
best_test2 = pd.concat(results2)

In [27]:
best_test2

,Model Type,Train RMSE,Cross-Val RMSE,Test RMSE,Train Accuracy,Test Accuracy,company
0,elastic net,-25.372072,-35.835940,-105.970960,0.511680,0.455556,GWW
1,SVR,-1.082459,-51.851582,-2.433871,0.503337,0.462222,GWW
2,Ridge,23.074238,-47.368749,92.372371,0.517798,0.460000,GWW
0,elastic net,-0.757683,-1.627002,-8.367635,0.526140,0.564444,FE
1,SVR,0.080416,-5.615035,-8.734714,0.491657,0.564444,FE
2,Ridge,0.722288,-1.576620,9.043852,0.521691,0.564444,FE
0,elastic net,-2.611824,-2.512044,-7.830670,0.553949,0.506667,TYL
1,SVR,0.997658,-3.160571,0.982852,0.483315,0.466667,TYL
2,Ridge,2.592848,-2.569836,8.533442,0.557842,0.466667,TYL
0,elastic net,-5.116733,-6.985517,-41.166376,0.510011,0.484444,RSG


In [23]:
best_test.sort_values('Test Accuracy', ascending = False)

,Model Type,Train RMSE,Cross-Val RMSE,Test RMSE,Train Accuracy,Test Accuracy,company
2,Ridge,1.795749,-2.407083,27.209715,0.475222,0.865140,TCYSF
1,SVR,0.649089,-2.672766,-7.513688,0.487294,0.865140,TCYSF
0,elastic net,-1.809677,-2.682003,-26.818113,0.435832,0.865140,TCYSF
1,SVR,0.985809,-2.688099,0.985862,0.536707,0.564444,WST
2,Ridge,0.722288,-1.576620,9.043852,0.521691,0.564444,FE
1,SVR,0.080416,-5.615035,-8.734714,0.491657,0.564444,FE
0,elastic net,-0.757683,-1.627002,-8.367635,0.526140,0.564444,FE
2,Ridge,1.531534,-1.445385,2.423286,0.544494,0.537778,EA
2,Ridge,1.531534,-1.445385,2.423286,0.544494,0.537778,EA
2,Ridge,20.809018,-47.084332,68.708145,0.499444,0.515556,FLT


In [ ]:
fig, ax = plt.subplots(figsize = (24,7))
plot_x = y_train.index
plot_y = (y_hat_train - y_train)/y_train
plt.scatter(plot_x,plot_y)
ax.yaxis.set_major_formatter('{x:1,.2%}')
plt.axhline(y=0, alpha = .5, color = 'red', linewidth = 4.0);

In [ ]:
fig2, ax = plt.subplots(figsize = (24,7))
ax.scatter(y_test.index, (y_hat_test - y_test)/y_test)
ax.yaxis.set_major_formatter('{x:1,.2%}')
plt.axhline(y=0, alpha = 1, color = 'red');

In [ ]:
fig3, ax = plt.subplots(figsize = (24,7))
fig3_data = preds_test[(preds_test.index >= '1-1-2020') & (preds_test.index <= '4-1-2021')]
ax.plot(fig3_data.adjustedclose, linewidth = 3.0)
ax.plot(fig3_data.predictions)
ax.yaxis.set_major_formatter('${x:1,.2f}');

In [ ]:
fig4, ax = plt.subplots(figsize = (24,7))
fig4_data = preds_train[(preds_train.index >= '1-1-2018') & (preds_train.index <= '1-1-2020')]
ax.plot(fig4_data.adjustedclose, linewidth = 3.0)
ax.plot(fig4_data.predictions)
ax.yaxis.set_major_formatter('${x:1,.2f}');

In [ ]:
#show prediction distance from previous close.